# Lab 3: Constituency parsing with CKY

The grammatical structure of a sentence can be represented with a Context Free Grammar (CFG). When we additionally assign probabilities to the rules of the CFG we get a PCFG: a _Probabilistic_ CFG. 

Given a sufficiently expressive PCFG (one that holds enough rules) we can parse new sentences using the Cocke–Kasami–Younger (CKY) algorithm. You can use this algorithm in three ways: to find the set of all the possible parses $p$ of a sentence $s$ under a PCFG $G$; to find the probability of the sentence by summing up the probabilities of these parses; or to find the parse $p^{*}$ of the highest probability.


### Tasks
1. In this notebook you will learn how to represent a PCFG in an object-oriented manner as a collection of python classes. These classes are already defined for you. Read them through thoroughly and make sure that you understand them well. You have to use them in task 2.

2. Implement the CKY algorithm to find the most probable parse $p^{*}$ for a sentence. Your implementation will follow the psuedo-code that is given in both the lecture slides, and Jurafsky and Martin.

# Rules
* The assignment should be submitted to **Blackboard** as `.ipynb` (without data files or zipping). Only **one submission per group**.

* The **filename** should be the group number, e.g., `01.ipynb` or `31.ipynb`.
 
* The questions marked **Extra** or **Optional** are an additional challenge for those interested in going the extra mile. There are no points for them.

**Rules for implementation**

* You should **write your code and answers in this iPython Notebook**. (See http://ipython.org/notebook.html for reference material.) If you have problems, please contact your teaching assistant.

* Use only **one cell for code** and **one cell for markdown** answers!    

    * Put all code in the cell with the `## YOUR CODE HERE ##` comment.
    * Provide brief comments on what the code does at crucial points.
    * For theoretical questions, put your solution in the `█████ YOUR ANSWER HERE █████` cell and keep the header(!).

* Don't change or delete any initially provided cells, either text or code, unless explicitly instructed to do so.
* Don't delete the comment lines `#TEST...` or edit their code cells. 
* Don't change the names of provided functions and variables or arguments of the functions. 
* Leave the output of your code in the output cells.
* Don't output unnecessary info (e.g., printing variables for debugging purposes). This clutters the notebook and slows down the grading.
* Test your code and **make sure we can run your notebook** in the colab environment.

<font color="red">You following these rules helps us to grade the submissions relatively efficiently. If these rules are violated, a submission will be subject to penalty points.</font>  

# <font color="red">Contributions</font>

~~Delete this text and write instead of it your:~~
* ~~a list of group members names (NOT student IDs)~~
* ~~who contributed to which exercises (you don't need to be very detailed)~~ 

# Set up

In [2]:
import numpy as np
from collections import Counter, defaultdict
import math
# nltk will be used to draw constituency parses
import nltk 
from nltk.tree import Tree

In [3]:
# downloading grammar files
! wget -nv https://naturallogic.pro/_files_/download/mNLP/groucho-grammar-1.txt
! wget -nv https://naturallogic.pro/_files_/download/mNLP/groucho-grammar-2.txt
! wget -nv https://naturallogic.pro/_files_/download/mNLP/telescope-grammar.txt

2023-06-12 16:22:14 URL:https://naturallogic.pro/_files_/download/mNLP/groucho-grammar-1.txt [337/337] -> "groucho-grammar-1.txt" [1]
2023-06-12 16:22:14 URL:https://naturallogic.pro/_files_/download/mNLP/groucho-grammar-2.txt [337/337] -> "groucho-grammar-2.txt" [1]
2023-06-12 16:22:15 URL:https://naturallogic.pro/_files_/download/mNLP/telescope-grammar.txt [381/381] -> "telescope-grammar.txt" [1]


# PCFG

In this lab we will show you a way to represent a **PCFG** using python objects. We will introduce the following classes:

* Symbol
    * Terminal
    * Nonterminal
* Rule

At first glance, this might seem like a lot of work. But, hopefully, by the time you get to implementing CKY you will be convinced in the benefits of these constructions.

## Symbol

Recall that:
* **Terminal** symbols are the words of the sentence: _I, ate, salad, the_ etc.
* **Nonterminal** symbols are the syntactic categories of the various constituents: _S, NP, VP, Det_ etc.

In our representation, `Symbol` is going to be a container class. The classes `Terminal` and `Nonterminal` will *inherit* from the `Symbol` class and will hence both become a type of symbol. The classes themselves are effectively a container for the underlying python strings. 

In [4]:
class Symbol:
    """
    A symbol in a grammar.
    This class will be used as parent class for Terminal, Nonterminal.
    This way both will be a type of Symbol.
    """
    def __init__(self):
        pass


class Terminal(Symbol):
    """
    Terminal symbols are words in a vocabulary
    
    E.g. 'I', 'ate', 'salad', 'the'
    """

    def __init__(self, symbol: str):
        assert type(symbol) is str, f"A Terminal takes a python string, got {type(symbol)}" 
        self._symbol = symbol

    def is_terminal(self):
        return True

    def is_nonterminal(self):
        return False

    def __str__(self):
        return f"'{self._symbol}'" 

    def __repr__(self):
        return f"Terminal({repr(self._symbol)})" 

    def __hash__(self):
        return hash(self._symbol)

    def __len__(self):
        """The length of the underlying python string"""
        return len(self._symbol)

    def __eq__(self, other):
        return type(self) == type(other) and self._symbol == other._symbol

    def __ne__(self, other):
        return not (self == other)

    def __lt__(self, other):
        return self._symbol < other._symbol
    
    @property
    def obj(self):
        """Returns the underlying python string"""
        return self._symbol


class Nonterminal(Symbol):
    """
    Nonterminal symbols are the grammatical classes in a grammar.
    
    E.g. S, NP, VP, N, Det, etc.
    """

    def __init__(self, symbol: str):
        assert type(symbol) is str, f"A Nonterminal takes a python string, got {type(symbol)}" 
        self._symbol = symbol

    def is_terminal(self):
        return False
    
    def is_nonterminal(self):
        return True

    def __str__(self):
        return f"[{self._symbol}]" 

    def __repr__(self):
        return f"Nonterminal({repr(self._symbol)})" 

    def __hash__(self):
        return hash(self._symbol)
    
    def __len__(self):
        """The length of the underlying python string"""
        return len(self._symbol)
    
    def __eq__(self, other):
        return type(self) == type(other) and self._symbol == other._symbol

    def __ne__(self, other):
        return not (self == other)

    def __lt__(self, other):
        return self._symbol < other._symbol
    
    @property
    def obj(self):
        """Returns the underlying python string"""
        return self._symbol

Let's try out the classes by initializing some terminal and nonterminal symbols:

In [5]:
dog = Terminal('dog')
the = Terminal('the')
walks = Terminal('walks')

S = Nonterminal('S')
NP = Nonterminal('NP')
NP_prime = Nonterminal('NP')
VP = Nonterminal('VP')
V = Nonterminal('V')
N = Nonterminal('N')
Det = Nonterminal('Det')

The methods `__eq__` and `__ne__` make it possible to compare our objects using standard Python syntax. But more importantly: compare in the way that we are interested in, namely whether the underlying representation is the same.

To see the difference, try commenting out the method `__eq__` in the class above, and notice the different result of the equality test `NP==NP_prime`.

In [6]:
print(dog)
print(NP)
print()
print(NP==Det)
print(NP!=Det)
print(NP==NP)
print(NP==NP_prime)

'dog'
[NP]

False
True
True
True


dNote the difference between calling `print(NP)` and simply calling `NP`. The first is taken care of by the method `__str__` and the second by the method `__repr__`.

In [7]:
dog

Terminal('dog')

We can also easily check if our symbol is a terminal or not:

In [8]:
dog.is_terminal()

True

In [9]:
NP.is_terminal()

False

Finally the method `__hash__` makes our object *hashable*, and hence usable in a datastructure like a dictionary. 

Try commenting out this method above in the class and then retry constructing the dictionary: notice the error.

In [10]:
d = {NP: 1, S: 2}
d

{Nonterminal('NP'): 1, Nonterminal('S'): 2}

## Rules

In a PCFG a **rule** looks something like this 

$$NP \to Det\;N$$

with a corresponding probability, for example $1.0$ if we lived in a world where all noun phrases had this grammatical structure.

In our representation, `Rule` will be an object made of a left-hand side (`lhs`) symbol, a sequence of right-hand side symbols (`rhs`) and a probability `prob`. 

If we use the above defined symbols, we can call

    rule = Rule(NP, [Det, N], 1.0)
   
This will construct an instance called `rule` which represent the rule above

    [NP] -> [Det] [N] (1.0)
    

In [11]:
class Rule:

    def __init__(self, lhs, rhs, prob):
        """
        Constructs a Rule.
        A Rule takes a LHS symbol and a list/tuple of RHS symbols.

        :param lhs: the LHS nonterminal
        :param rhs: a sequence of RHS symbols (terminal or nonterminal)
        :param prob: probability of the rule
        """

        assert isinstance(lhs, Symbol), 'LHS must be an instance of Symbol (actually even a non-terminal but later we will expan LHS)'
        assert len(rhs) > 0, 'If you want an empty RHS, use an epsilon Terminal EPS'
        assert all(isinstance(s, Symbol) for s in rhs), 'RHS must be a sequence of Symbol objects'
        if prob is not None:
            assert 0 <= prob <= 1, 'The probability must be between 0 and 1'
        self._lhs = lhs
        self._rhs = tuple(rhs)
        self._prob = prob


    def __eq__(self, other):
        return self._lhs == other._lhs and self._rhs == other._rhs and self._prob == other._prob

    def __ne__(self, other):
        return not (self == other)

    def __hash__(self):
        return hash((self._lhs, self._rhs, self._prob))

    def __repr__(self):
        rhs = ' '.join(str(sym) for sym in self._rhs)
        return f"{self._lhs} -> {rhs} ({self.prob})"

    def is_binary(self):
        """True if Rule is binary: A -> B C"""
        return len(self._rhs) == 2
    
    def is_unary(self):
        """True if Rule is unary: A -> w"""
        return len(self._rhs) == 1
    
    @property
    def lhs(self):
        """Returns the lhs of the rule"""
        return self._lhs

    @property
    def rhs(self):
        """Returns the rhs of the rule"""
        return self._rhs

    @property
    def prob(self):
        """Returns the probability of the rule"""
        return self._prob


Just as with `Terminal` and `Nonterminal` you can print an instance of `Rule`, you can access its attributes, and you can hash rules with containers such as dict and set.

In [12]:
r1 = Rule(S, [NP, VP], 1.0)
r2 = Rule(NP, [Det, N], 1.0)
r3 = Rule(N, [dog], 1.0)
r4 = Rule(Det, [the], 1.0)
r5 = Rule(VP, [walks], 1.0)

print(r1)
print(r2)
print(r3)
print(r4)

[S] -> [NP] [VP] (1.0)
[NP] -> [Det] [N] (1.0)
[N] -> 'dog' (1.0)
[Det] -> 'the' (1.0)


In [13]:
print(r1.prob)

1.0


In [14]:
r1 in set([r1])

True

In [15]:
d = {r1: 1, r2: 2}
d

{[S] -> [NP] [VP] (1.0): 1, [NP] -> [Det] [N] (1.0): 2}

## Grammar

A `PCFG` class is a container for `Rules`. The `Rules` are stored in the `PCFG` in such a way that they can be accesed easily in different ways.

In [16]:
class PCFG(object):
    """
    Constructs a PCFG.
    A PCFG stores a list of rules that can be accessed in various ways.
    
    :param rules: an optional list of rules to initialize the grammar with
    """
    def __init__(self, rules=[]):
        self._rules = []
        self._rules_by_lhs = defaultdict(list)
        self._terminals = set()
        self._nonterminals = set()
        for rule in rules:
            self.add(rule)

    def add(self, rule):
        """Adds a rule to the grammar"""
        if not rule in self._rules:
            self._rules.append(rule)
            self._rules_by_lhs[rule.lhs].append(rule)
            self._nonterminals.add(rule.lhs)
            for s in rule.rhs:
                if s.is_terminal():
                    self._terminals.add(s)
                else:
                    self._nonterminals.add(s)

    def update(self, rules):
        """Add a list of rules to the grammar"""
        for rule in rules:
            self.add(rule)

    @property
    def nonterminals(self):
        """The list of nonterminal symbols in the grammar"""
        return self._nonterminals

    @property
    def terminals(self):
        """The list of terminal symbols in the grammar"""
        return self._terminals
    
    @property
    def rules(self):
        """The list of rules in the grammar"""
        return self._rules
    
    @property
    def binary_rules(self):
        """The list of binary rules in the grammar"""
        return [rule for rule in self._rules if rule.is_binary()]
    
    @property
    def unary_rules(self):
        """The list of unary rules in the grammar"""
        return [rule for rule in self._rules if rule.is_unary()]

    def __len__(self):
        return len(self._rules)

    def get(self, lhs):
        """The list of rules whose LHS is the given symbol lhs"""
        return self._rules_by_lhs.get(lhs, [])

    def __iter__(self):
        """Iterator over rules (in arbitrary order)"""
        return iter(self._rules)

    def iteritems(self):
        """Iterator over pairs of the kind (LHS, rules rewriting LHS)"""
        return self._rules_by_lhs.items()

    def __str__(self):
        """Prints the grammar line by line"""
        lines = []
        for lhs, rules in self.iteritems():
            for rule in rules:
                lines.append(str(rule))
        return '\n'.join(lines)

Initialize a grammar

In [17]:
G = PCFG()

We can add rules individually with `add`, or as a list with `update`:

In [18]:
G.add(r1)
G.update([r2,r3,r4,r5])

We can print the grammar

In [19]:
print(G)

[S] -> [NP] [VP] (1.0)
[NP] -> [Det] [N] (1.0)
[N] -> 'dog' (1.0)
[Det] -> 'the' (1.0)
[VP] -> 'walks' (1.0)


We can get the set of rewrite rules for a certain LHS symbol.

In [20]:
G.get(S)

[[S] -> [NP] [VP] (1.0)]

In [21]:
G.get(NP)

[[NP] -> [Det] [N] (1.0)]

We can also iterate through rules in the grammar.

Note that the following is basically counting how many rules we have in the grammar.

In [22]:
sum(1 for r in G)

5

which can also be done in a more efficient way

In [23]:
len(G)

5

We can access the set of terminals and nonterminals of the grammar:

In [24]:
print(G.nonterminals)

{Nonterminal('N'), Nonterminal('VP'), Nonterminal('Det'), Nonterminal('S'), Nonterminal('NP')}


In [25]:
print(G.terminals)

{Terminal('dog'), Terminal('walks'), Terminal('the')}


In [26]:
S in G.nonterminals

True

In [27]:
dog in G.terminals

True

Finally we can easily access all the binary rules and all the unary rules in the grammar:

In [28]:
G.unary_rules

[[N] -> 'dog' (1.0), [Det] -> 'the' (1.0), [VP] -> 'walks' (1.0)]

In [29]:
G.binary_rules

[[S] -> [NP] [VP] (1.0), [NP] -> [Det] [N] (1.0)]

In [30]:
def get_nonterminals_for_word(grammar, word):
    nonterminals = {}
    
    for rule in grammar.rules:

        if str(rule.rhs[0]) == str(Terminal(word)):
            nonterminal = rule.lhs 
            nonterminals[nonterminal] = True

    return nonterminals

In [31]:
get_nonterminals_for_word(G, 'the')

{Nonterminal('Det'): True}

# Visualizing a tree

For the sake of legacy let's reiterate an age-old NLP schtick, the well-known example of structural ambiguity from the Groucho Marx movie, [Animal Crackers](https://youtu.be/FZUfhfHbjE4?t=1m33s) (1930):

> One morning I shot an elephant in my pajamas. How he got into my pajamas, I don't know.

Let's take a closer look at the ambiguity in the phrase: _I shot an elephant in my pajamas_. The ambiguity is caused by the fact that the sentence has two competing parses represented in:

    (S (NP I) (VP (VP (V shot) (NP (Det an) (N elephant))) (PP (P in) (NP (Det my) (N pajamas)))))

and

    (S (NP I) (VP (V shot) (NP (Det an) (NP (N elephant) (PP (P in) (NP (Det my) (N pajamas)))))))


We can write these parses down as strings and then let NLTK turn them into trees using the NLTK `Tree` class. (See http://www.nltk.org/api/nltk.html#nltk.tree.Tree as reference for this class, if you want to know more.)

In [32]:
parse1 = "(S (NP I) (VP (VP (V shot) (NP (Det an) (N elephant))) (PP (P in) (NP (Det my) (N pajamas)))))"
parse2 = "(S (NP I) (VP (V shot) (NP (Det an) (NP (N elephant) (PP (P in) (NP (Det my) (N pajamas)))))))"

pajamas1 = Tree.fromstring(parse1)
pajamas2 = Tree.fromstring(parse2)

In [33]:
print(pajamas1)

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))


We can then *pretty-print* these trees:

In [34]:
pajamas1.pretty_print()
pajamas2.pretty_print()

     S                                       
  ___|______________                          
 |                  VP                       
 |         _________|__________               
 |        VP                   PP            
 |    ____|___              ___|___           
 |   |        NP           |       NP        
 |   |     ___|_____       |    ___|_____     
 NP  V   Det        N      P  Det        N   
 |   |    |         |      |   |         |    
 I  shot  an     elephant  in  my     pajamas

     S                                       
  ___|__________                              
 |              VP                           
 |    __________|______                       
 |   |                 NP                    
 |   |     ____________|___                   
 |   |    |                NP                
 |   |    |      __________|___               
 |   |    |     |              PP            
 |   |    |     |       _______|___           
 |   |    |     |      

# Parsing with CKY

Let's stick with this sentence for the rest of this lab. We will use CKY to find the 'best' parse for this sentence.

In [35]:
# Turn the sentence into a list
sentence = "I shot an elephant in my pajamas".split()
# The length of the sentence
num_words = len(sentence)

A PCFG for this sentence can be found in the file `groucho-grammar-1.txt`. We read this in with the function `read_grammar_rules`.

In [36]:
def read_grammar_rules(istream):
    """Reads grammar rules formatted as 'LHS ||| RHS ||| PROB'."""
    for line in istream:
        line = line.strip()
        if not line: continue
        fields = line.split('|||')
        if len(fields) != 3:
            raise ValueError(f"Three fields were expected: {fields}")
        lhs = fields[0].strip()

        if lhs.startswith('[') and lhs.endswith(']'):
            lhs = Nonterminal(lhs[1:-1])
        else:
            raise ValueError(f"LHS must be a non-terminal: {fields}")
        rhs = fields[1].strip().split()
        new_rhs = []
        for r in rhs:
            if r.startswith('[') and r.endswith(']'):
                r = Nonterminal(r[1:-1])
            else:
                r = Terminal(r)
            new_rhs.append(r)

        prob = float(fields[2].strip())
        yield Rule(lhs, new_rhs, prob)

In [37]:
# Read in the grammar
with open('groucho-grammar-1.txt') as F:
    grammar = PCFG(read_grammar_rules(F))
print("The grammar:\n", grammar)

The grammar:
 [S] -> [NP] [VP] (1.0)
[PP] -> [P] [NP] (1.0)
[NP] -> [Det] [N] (0.2)
[NP] -> [Det] [NP] (0.3)
[NP] -> [N] [PP] (0.3)
[NP] -> 'I' (0.2)
[VP] -> [V] [NP] (0.4)
[VP] -> [VP] [PP] (0.6)
[Det] -> 'an' (0.6)
[Det] -> 'my' (0.4)
[N] -> 'elephant' (0.5)
[N] -> 'pajamas' (0.5)
[V] -> 'shot' (1.0)
[P] -> 'in' (1.0)


In [38]:
[rule.prob for rule in grammar.get(S)]



[1.0]

In [39]:
X = get_nonterminals_for_word(grammar, 'pajamas')
[key for key in X.keys()]

[Nonterminal('N')]

In [40]:
for word in sentence: 
  for rule in grammar.rules: 
    if word in rule.rhs: 
      print(rule.rhs)

# In the line for A in grammar.rules_rhs[word]:, the code is iterating over the non-terminal symbols (A) that can produce the current word (word) according to the grammar rules.

We will also need the following two dictionaries: `nonterminal2index` mapping from nonterminals to integers (indices); and its inverse, an `index2nonterminal` dictionary.

In [41]:
num_nonterminals = len(grammar.nonterminals)

# Make a nonterminal2index and a index2nonterminal dictionary
n2i = defaultdict(lambda: len(n2i))
i2n = dict()

# sort nonterminals to make the mapping deterministic
for nt in sorted(grammar.nonterminals):
    i2n[n2i[nt]] = nt

# Stop defaultdict behavior of n2i
n2i = dict(n2i)

n2i

{Nonterminal('Det'): 0,
 Nonterminal('N'): 1,
 Nonterminal('NP'): 2,
 Nonterminal('P'): 3,
 Nonterminal('PP'): 4,
 Nonterminal('S'): 5,
 Nonterminal('V'): 6,
 Nonterminal('VP'): 7}

## The charts

Now we are ready to introduce the chart datastructures. We need a chart to store the **scores** and a chart to store the **backpointers**.

Both of these will be 3-dimensional numpy arrays: one named `score` holding the probabilities of intermediate results; one named `back` to store the backpointers in. We will use the following indexing convention for these charts:

* Format of the chart holding the **scores** `score[A][begin][end] = probability`.
This is interpreted as the probability of the constituent between `begin:end` being parsed with `A` as its root.      
         
* Format of the chart holding the **backpointers** `back[A][begin][end] = (split, B, C)`. 
This is interpreted as the constituent `begin:end` can be combined with a rule `A -> B C` where `begin:split` is `B` and `split:end` is `C`.

This indexing convention is convenient for printing. See what happens when we print `back` below: we get `num_nonterminal` slices, each a numpy array of shape `[n_words+1, n_words+1]`. This is easier to read than the format `score[i][j][A]`.

**[Note]** Here we pretended `A` is both the nonterminal as well as the index. In our implementation `A` will be the nonterminal and the index for `A` will be `n2i[A]`.

Let's show you what we mean:

In [42]:
# A numpy array zeros
score = np.zeros((num_nonterminals,
                  num_words + 1, 
                  num_words + 1))

# A numpy array that can store arbitrary data (we set dtype to object)
back = np.zeros((num_nonterminals,
                 num_words + 1, 
                 num_words + 1), dtype=object)

The following illustrates the way you will use the `back` chart. In this example, your parser recognized that the entire sequence is S while the words between 0 and 2 form NP, and the words between 2 and the end of the sentence form VP (and nothing else yet):

In [43]:
# Illustration of the backpointer array
back[n2i[S]][0][-1] = (2,NP,VP) 
back

array([[[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        

## Ex1 [60pt] CKY parsing

Implement the **CKY** algorithm. Follow the pseudo-code given in the lecture-slides (or alternatively in J&M). The code must comply to the following:

* The function `cky` takes a sentence (list of words), a grammar (an instance of PCFG), and a n2i non-terminals-to-index dictionary.
* The function `cky` returns the filled-in score-chart and backpointer-chart, following the format established above.
* No global variables should be accessed from the body of the function (except for the predefined classes).

**[Hint]** This is the moment to make good use of the methods of the classes `PCFG`, `Rule`, `Nonterminal`, and `Terminal`!

In [44]:
from numpy.lib.function_base import kaiser
#MY FUNCTION
def cky(sentence, grammar, n2i):
    """
    The CKY algorithm.
    
    :param sentence: a list of words
    :param grammar: an instance of the class PCFG
    :param n2i: a dictionary mapping from Nonterminals to indices
    :return score: the filled in scores chart
    :return back: the filled in backpointers chart 
    """
    num_words = len(sentence)
    num_nonterminals = len(grammar.nonterminals)
    
    # A numpy array to store the scores of intermediate parses
    score = np.zeros((num_nonterminals,
                  num_words + 1, 
                  num_words + 1))

    # A numpy array to store the backpointers
    back = np.zeros((num_nonterminals,
                     num_words + 1, 
                     num_words + 1), dtype=object)
    # Initialize an empty table that will store the parsing information.

    for j in range(num_words):
      for A in grammar.nonterminals: 
        rules = grammar.get(A)
        for rule in rules: 
          if rule.is_unary() == True and str(rule.rhs[0]) == str(Terminal(sentence[j])): #NK change check to is_unary
                  back[n2i[A]][j][j] = A

                  score[n2i[A]][j][j] = rule.prob 


    # Fill in the upper triangular part of the CYK table
    for length in range(2, num_words+1 ):
        for i in range(num_words - length +1):
            j = i + length
            for k in range(i, j):
                print(sentence[k])

                B = back[:, i, j - length]
                C = back[:, k, j-1, ]

                B_score = score[:,i, j - length, ]
                C_score = score[:, k, j-1]

    
                print("B", str(B))
                print("C", str(C))

                print("B_score", str(sum(B_score)))
                print("C_score", str(sum(C_score)))


                for A in grammar.nonterminals: 
                    rules = grammar.get(A)

                    for rule in rules: 
                      if rule.is_binary(): 
                      
                        # if (rule.rhs[0] in B) and (rule.rhs[1] in C):
                        if np.any(B == rule.rhs[0]) and np.any(C == rule.rhs[1]):

                          print(rule)
   
                          A = rule.lhs
                          back[n2i[A]][i][j] = A
                          
                          #Calculate probability by multiplyiing probabilities of previous cells
                          print("CALCULATION:", sum(B_score), "*", sum(C_score), "*", rule.prob, "=", sum(B_score)* sum(C_score) * rule.prob)
                          
                          score[n2i[A]][i][j] = sum(B_score) * sum(C_score) * rule.prob
       

            print("------------------------")

        print("------------------------")





    return score, back

In [45]:
# Run CKY
score, back = cky(sentence, grammar, n2i)

print(np.array(score))
print(np.array(back))



I
B [0 0 Nonterminal('NP') 0 0 0 0 0]
C [0 0 0 0 0 0 0 0]
B_score 0.2
C_score 0.0
shot
B [0 0 Nonterminal('NP') 0 0 0 0 0]
C [0 0 0 0 0 0 Nonterminal('V') 0]
B_score 0.2
C_score 1.0
------------------------
shot
B [0 0 0 0 0 0 Nonterminal('V') 0]
C [0 0 0 0 0 0 0 0]
B_score 1.0
C_score 0.0
an
B [0 0 0 0 0 0 Nonterminal('V') 0]
C [Nonterminal('Det') 0 0 0 0 0 0 0]
B_score 1.0
C_score 0.6
------------------------
an
B [Nonterminal('Det') 0 0 0 0 0 0 0]
C [0 0 0 0 0 0 0 0]
B_score 0.6
C_score 0.0
elephant
B [Nonterminal('Det') 0 0 0 0 0 0 0]
C [0 Nonterminal('N') 0 0 0 0 0 0]
B_score 0.6
C_score 0.5
[NP] -> [Det] [N] (0.2)
CALCULATION: 0.6 * 0.5 * 0.2 = 0.06
------------------------
elephant
B [0 Nonterminal('N') 0 0 0 0 0 0]
C [0 0 0 0 0 0 0 0]
B_score 0.5
C_score 0.0
in
B [0 Nonterminal('N') 0 0 0 0 0 0]
C [0 0 0 Nonterminal('P') 0 0 0 0]
B_score 0.5
C_score 1.0
------------------------
in
B [0 0 0 Nonterminal('P') 0 0 0 0]
C [0 0 0 0 0 0 0 0]
B_score 1.0
C_score 0.0
my
B [0 0 0 Nonterm

### Check your CKY

Use the code in the following two cell to check your `cky` implementation.

Take the Nonterminal `S` to inspect your filled in score and backpointer charts. **Leave the code in this cell unchanged.** We will use this to evaluate the corectness of your cky function.

In [46]:
# TEST EX1

S = Nonterminal('S')

print('The whole slice for nonterminal S:')
print(score[n2i[S]], "\n")

print('The score in cell (S, 0, num_words), which is the probability of the best parse:')
print(score[n2i[S]][0][num_words], "\n")


print('The backpointer in cell (S, 0, num_words):')
print(back[n2i[S]][0][num_words], "\n")

The whole slice for nonterminal S:
[[0.       0.       0.       0.       0.       0.       0.0048   0.000864]
 [0.       0.       0.       0.       0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.       0.       0.       0.      ]] 

The score in cell (S, 0, num_words), which is the probability of the best parse:
0.0008640000000000001 

The backpointer in cell (S, 0, num_words):
[S] 



### Alternative Groucho-grammar

### The man with the telescope

Another ambiguous sentence:

> I saw the man on the hill with the telescope.

A grammar for this sentence is specified in the file `telescope-grammar.txt`.

In [62]:
## YOUR CODE HERE ##
# Read in the grammar
with open('telescope-grammar.txt') as F:
    grammar = PCFG(read_grammar_rules(F))
print("The grammar:\n", grammar)


num_nonterminals = len(grammar.nonterminals)

# Make a nonterminal2index and a index2nonterminal dictionary
n2i = defaultdict(lambda: len(n2i))
i2n = dict()

# Turn the sentence into a list
sentence = "I saw the man on the hill with the telescope".split()
# The length of the sentence
num_words = len(sentence)

# sort nonterminals to make the mapping deterministic
for nt in sorted(grammar.nonterminals):
    i2n[n2i[nt]] = nt

# Stop defaultdict behavior of n2i
n2i = dict(n2i)

#Run CYK algorithm
score, back = cky(sentence, grammar, n2i)
# print(np.array(score))
# print(np.array(back))


#Test code
S = Nonterminal('S')

print('The whole slice for nonterminal S:')
print(score[n2i[S]], "\n")

print('The score in cell (S, 0, num_words), which is the probability of the best parse:')
print(score[n2i[S]][0][num_words], "\n")


print('The backpointer in cell (S, 0, num_words):')
print(back[n2i[S]][0][num_words], "\n")

The grammar:
 [S] -> [NP] [VP] (1.0)
[VP] -> [V] [NP] (0.6)
[VP] -> [VP] [PP] (0.4)
[NP] -> [NP] [PP] (0.3)
[NP] -> [Det] [N] (0.3)
[NP] -> [Det] [NP] (0.2)
[NP] -> 'I' (0.2)
[PP] -> [P] [NP] (0.8)
[PP] -> [PP] [PP] (0.2)
[V] -> 'saw' (1.0)
[Det] -> 'the' (1.0)
[N] -> 'man' (0.4)
[N] -> 'hill' (0.3)
[N] -> 'telescope' (0.3)
[P] -> 'on' (0.5)
[P] -> 'with' (0.5)
I
B [0 0 Nonterminal('NP') 0 0 0 0 0]
C [0 0 0 0 0 0 0 0]
B_score 0.2
C_score 0.0
saw
B [0 0 Nonterminal('NP') 0 0 0 0 0]
C [0 0 0 0 0 0 Nonterminal('V') 0]
B_score 0.2
C_score 1.0
------------------------
saw
B [0 0 0 0 0 0 Nonterminal('V') 0]
C [0 0 0 0 0 0 0 0]
B_score 1.0
C_score 0.0
the
B [0 0 0 0 0 0 Nonterminal('V') 0]
C [Nonterminal('Det') 0 0 0 0 0 0 0]
B_score 1.0
C_score 1.0
------------------------
the
B [Nonterminal('Det') 0 0 0 0 0 0 0]
C [0 0 0 0 0 0 0 0]
B_score 1.0
C_score 0.0
man
B [Nonterminal('Det') 0 0 0 0 0 0 0]
C [0 Nonterminal('N') 0 0 0 0 0 0]
B_score 1.0
C_score 0.4
[NP] -> [Det] [N] (0.3)
CALCULATION: 